In [1]:
import cv2
import numpy as np
import pyautogui
import time
import os
from typing import List, Tuple, Dict, Optional
from enum import Enum
import threading
import json

class CardType(Enum):
    """卡牌类型枚举"""
    LIGHTNING = "lightning"  # 闪电
    SNOWFLAKE = "snowflake"  # 雪花
    FIRE = "fire"           # 火焰
    KETTLE = "kettle"       # 热水壶

class PlayerState(Enum):
    """玩家状态枚举"""
    NORMAL1 = "normal1"       # 正常
    DRUNK1 = "drunk1"         # 醉酒倒下
    NORMAL2 = "normal2"       
    DRUNK2 = "drunk2"         
    NORMAL3 = "normal3"       
    DRUNK3 = "drunk3"         


In [101]:
class GameAutomation:
    """游戏自动化主类"""
    
    def __init__(self):
        # 初始化配置
        self.config = {
            'screen_width': 1920,
            'screen_height': 1080,
            'card_templates_dir': 'card_templates',
            'player_templates_dir': 'player_templates',
            'confidence_threshold': 0.7,
            'detection_interval': 0.5,  # 检测间隔（秒）
        }
        
        # 游戏区域配置
        self.game_areas = {
            'target_card_area': (27, 885, 105, 990),  # 左下角目标卡牌区域 (x1, y1, x2, y2)
            'hand_cards_area': (540, 625, 1356, 906),  # 屏幕中央手牌区域
            'player_positions': {
                'player1': (0, 240, 430, 870),   # 玩家1位置
                'player2': (810, 200, 1150, 610), # 玩家2位置
                'player3': (1495, 190, 1925, 790),   # 玩家3位置
            },
            'confirm_position':(1760,955)
        }
        
        # 存储模板图像
        self.card_templates = {}
        self.player_templates = {}
        
        # 当前游戏状态
        self.current_state = {
            'target_card': None,
            'hand_cards': [],
            'player_states': {},
            'is_my_turn': False
        }
        
        # 初始化PyAutoGUI
        pyautogui.FAILSAFE = True
        pyautogui.PAUSE = 0.1
        
        # 加载模板图像
        self.load_templates()
        
    def load_templates(self):
        """加载卡牌和玩家状态模板图像"""
        try:
            # 加载卡牌模板
            for card_type in CardType:
                template_path = os.path.join(self.config['card_templates_dir'], f'{card_type.value}.png')
                if os.path.exists(template_path):
                    self.card_templates[card_type] = cv2.imread(template_path, cv2.IMREAD_COLOR)
                    print(f"已加载卡牌模板: {card_type.value}")
                else:
                    print(f"警告: 找不到卡牌模板 {template_path}")
            
            # 加载玩家状态模板
            for state in PlayerState:
                template_path = os.path.join(self.config['player_templates_dir'], f'{state.value}.png')
                if os.path.exists(template_path):
                    self.player_templates[state] = cv2.imread(template_path, cv2.IMREAD_COLOR)
                    print(f"已加载玩家状态模板: {state.value}")
                else:
                    print(f"警告: 找不到玩家状态模板 {template_path}")
                    
        except Exception as e:
            print(f"加载模板时出错: {e}")
    
    def capture_screen(self) -> np.ndarray:
        """截取屏幕图像"""
        try:
            screenshot = pyautogui.screenshot()
            return cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"截屏失败: {e}")
            return None
    
    def template_match(self, image: np.ndarray, template: np.ndarray, 
                      threshold: float = None) -> Tuple[bool, Tuple[int, int], float]:
        """模板匹配函数"""
        if threshold is None:
            threshold = self.config['confidence_threshold']
            
        # 执行模板匹配
        result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # 判断是否匹配成功
        if max_val >= threshold:
            return True, max_loc, max_val
        else:
            return False, None, max_val
    
    def detect_target_card(self, screen: np.ndarray) -> Optional[CardType]:
        """检测左下角目标卡牌（无匹配时返回置信度最高项）"""
        x1, y1, x2, y2 = self.game_areas['target_card_area']
        target_area = screen[y1:y2, x1:x2]
        best_match = None
        best_confidence = 0
        all_matches = []  # 新增：存储所有匹配结果[2,4](@ref)
        
        for card_type, template in self.card_templates.items():
            if template is None:
                continue
                
            matched, position, confidence = self.template_match(target_area, template)
            all_matches.append((card_type, confidence))  # 记录所有结果
            
            if confidence > best_confidence:  # 始终更新最佳置信度
                best_match = card_type
                best_confidence = confidence
        
        # 无完美匹配时选择置信度最高项（即使低于阈值）
        if best_match:
            print(f"检测到目标卡牌: {best_match.value} (置信度: {best_confidence:.2f})")
        else:
            # 从所有结果中筛选最佳（可能包含低置信度匹配）
            best_match = max(all_matches, key=lambda x: x[1])[0] if all_matches else None
            print(f"无精确匹配，采用最高置信度卡牌: {best_match.value} ({best_confidence:.2f})")
        
        return best_match
    
    def detect_player_states(self, screen: np.ndarray) -> Dict[str, PlayerState]:
        """检测三个玩家的状态"""
        player_states = {}
        
        # 定义每个玩家对应的状态模板
        player_templates_mapping = {
            'player1': {'normal': PlayerState.NORMAL1, 'drunk': PlayerState.DRUNK1},
            'player2': {'normal': PlayerState.NORMAL2, 'drunk': PlayerState.DRUNK2},
            'player3': {'normal': PlayerState.NORMAL3, 'drunk': PlayerState.DRUNK3}
        }
        
        for player_name, (x1, y1, x2, y2) in self.game_areas['player_positions'].items():
            player_area = screen[y1:y2, x1:x2]
            
            # 获取该玩家对应的状态模板
            player_template_states = player_templates_mapping[player_name]

            # 保存图片以供测试
            #cv2.imwrite(f"{player_name}_area_.png", player_area)
            #print(f"已保存玩家区域图像: {player_name}_area_.png")
            
            best_match = None
            best_confidence = 0
            
            # 检测该玩家的醉酒状态
            drunk_state = player_template_states['drunk']
            if drunk_state in self.player_templates:
                drunk_template = self.player_templates[drunk_state]
                matched, position, confidence = self.template_match(player_area, drunk_template)
                #print("醉酒匹配置信度: ",matched,confidence)
                if matched and confidence > best_confidence:
                    best_match = drunk_state
                    best_confidence = confidence
            
            # 检测该玩家的正常状态
            normal_state = player_template_states['normal']
            if normal_state in self.player_templates:
                normal_template = self.player_templates[normal_state]
                matched, position, confidence = self.template_match(player_area, normal_template)
                #print("正常匹配置信度: ",matched,confidence)
                
                if matched and confidence > best_confidence:
                    best_match = normal_state
                    best_confidence = confidence
            
            # 如果没有匹配到任何状态，默认为正常状态
            if best_match is None:
                best_match = player_template_states['normal']
                print(f"{player_name} 状态未识别，默认为正常状态")
            else:
                status = "醉酒倒下" if "drunk" in best_match.value else "正常"
                print(f"{player_name} 状态: {status} (置信度: {best_confidence:.2f})")
            
            player_states[player_name] = best_match
        
        return player_states
    
    def detect_hand_cards(self, screen: np.ndarray) -> List[Dict]:
        """检测屏幕中央的手牌"""
        x1, y1, x2, y2 = self.game_areas['hand_cards_area']
        hand_area = screen[y1:y2, x1:x2]
        
        detected_cards = []
        
        # 使用轮廓检测来分离重叠的卡牌
        gray = cv2.cvtColor(hand_area, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(f"wedge_.png", gray)
        
        # 应用阈值和形态学操作
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        kernel = np.ones((3, 3), np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
        
        # 查找轮廓
        contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # 过滤轮廓，找到可能的卡牌
        card_contours = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 500:  # 卡牌最小面积阈值
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / h
                if 0.6 < aspect_ratio < 1.0:  # 卡牌宽高比范围
                    card_contours.append((x, y, w, h))
        
        # 按x坐标排序（从左到右）
        card_contours.sort(key=lambda x: x[0])
        
        # 对每个检测到的卡牌区域进行类型识别
        for i, (x, y, w, h) in enumerate(card_contours):
            card_roi = hand_area[y:y+h, x:x+w]
            
            # 识别卡牌类型
            best_match = None
            best_confidence = 0
            
            for card_type, template in self.card_templates.items():
                if template is None:
                    continue
                    
                # 调整模板大小以匹配检测到的卡牌
                resized_template = cv2.resize(template, (w, h))
                matched, position, confidence = self.template_match(card_roi, resized_template, 0.6)
                
                if matched and confidence > best_confidence:
                    best_match = card_type
                    best_confidence = confidence
            
            if best_match:
                detected_cards.append({
                    'type': best_match,
                    'position': (x + x1, y + y1),  # 转换为屏幕坐标
                    'size': (w, h),
                    'confidence': best_confidence,
                    'index': i
                })
        
        print(f"检测到 {len(detected_cards)} 张手牌")
        for card in detected_cards:
            print(f"  卡牌 {card['index']}: {card['type'].value} (置信度: {card['confidence']:.2f})")
        
        return detected_cards
    
    def is_my_turn(self, screen: np.ndarray) -> bool:
        """判断是否轮到玩家回合"""
        # 检测屏幕中央是否有卡牌出现
        hand_cards = self.detect_hand_cards(screen)
        return len(hand_cards) > 0
    
    def decide_action(self, target_card: CardType, hand_cards: List[Dict], 
                     player_states: Dict[str, PlayerState]) -> Dict:
        """决策要执行的动作"""
        # 这里实现你的游戏逻辑
        # 根据目标卡牌和手牌决定要打出什么牌
        
        action = {
            'type': 'play_card',
            'card_index': None,
            'target_player': None
        }
        
        # 简单的决策逻辑示例
        if target_card and hand_cards:
            # 寻找匹配的卡牌
            for card in hand_cards:
                if card['type'] == target_card:
                    action['card_index'] = card['index']
                    action['card_position'] = card['position']
                    action['card_size'] = card['size']
                    break
            
            # 如果没有匹配的卡牌，选择第一张
            if action['card_index'] is None and hand_cards:
                first_card = hand_cards[0]
                action['card_index'] = first_card['index']
                action['card_position'] = first_card['position']
                action['card_size'] = first_card['size']
        
        return action
    
    def execute_action(self, action: Dict):
        """执行游戏动作"""
        if action['type'] == 'play_card' and action['card_index'] is not None:
            # 点击选中的卡牌
            card_pos = action['card_position']
            card_size = action['card_size']
            
            # 点击卡牌中心
            click_x = card_pos[0] + card_size[0] // 2
            click_y = card_pos[1] + card_size[1] // 2
            
            print(f"点击卡牌 {action['card_index']} 位置: ({click_x}, {click_y})")
            
            # 执行点击
            pyautogui.click(click_x, click_y)
            time.sleep(0.5)
            
            # 点击确认按钮
            confirm_x,confirm_y = self.game_areas['confirm_position']
            
            print(f"点击确认按钮位置: ({confirm_x}, {confirm_y})")
            pyautogui.click(confirm_x, confirm_y)
    
    def run_detection_loop(self):
        """主要的检测循环"""
        print("开始游戏检测循环...")
        
        while True:
            try:
                # 截取屏幕
                screen = self.capture_screen()
                if screen is None:
                    time.sleep(self.config['detection_interval'])
                    continue
                
                # 检测目标卡牌
                target_card = self.detect_target_card(screen)
                self.current_state['target_card'] = target_card
                
                # 检测玩家状态
                player_states = self.detect_player_states(screen)
                self.current_state['player_states'] = player_states
                
                # 检测是否轮到玩家回合
                is_my_turn = self.is_my_turn(screen)
                self.current_state['is_my_turn'] = is_my_turn
                
                if is_my_turn:
                    # 检测手牌
                    hand_cards = self.detect_hand_cards(screen)
                    self.current_state['hand_cards'] = hand_cards
                    
                    # 决策动作
                    action = self.decide_action(target_card, hand_cards, player_states)
                    
                    # 执行动作
                    if action['card_index'] is not None:
                        self.execute_action(action)
                        time.sleep(2)  # 等待动作完成
                
                # 等待下一次检测
                time.sleep(self.config['detection_interval'])
                
            except KeyboardInterrupt:
                print("检测循环已停止")
                break
            except Exception as e:
                print(f"检测循环中出现错误: {e}")
                time.sleep(1)
    
    def create_template_directories(self):
        """创建模板目录"""
        os.makedirs(self.config['card_templates_dir'], exist_ok=True)
        os.makedirs(self.config['player_templates_dir'], exist_ok=True)
        print(f"已创建模板目录: {self.config['card_templates_dir']}, {self.config['player_templates_dir']}")
    
    def save_current_state(self, filename: str = 'game_state.json'):
        """保存当前游戏状态"""
        state_data = {
            'target_card': self.current_state['target_card'].value if self.current_state['target_card'] else None,
            'hand_cards_count': len(self.current_state['hand_cards']),
            'player_states': {k: v.value for k, v in self.current_state['player_states'].items()},
            'is_my_turn': self.current_state['is_my_turn']
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(state_data, f, ensure_ascii=False, indent=2)
        
        print(f"游戏状态已保存到: {filename}")

In [102]:
game_auto = GameAutomation()

已加载卡牌模板: lightning
已加载卡牌模板: snowflake
已加载卡牌模板: fire
已加载卡牌模板: kettle
已加载玩家状态模板: normal1
已加载玩家状态模板: drunk1
已加载玩家状态模板: normal2
已加载玩家状态模板: drunk2
已加载玩家状态模板: normal3
已加载玩家状态模板: drunk3


In [89]:
game_auto.create_template_directories()

已创建模板目录: card_templates, player_templates


In [97]:
time.sleep(3)
screen = game_auto.capture_screen()
print("截屏成功")

截屏成功


In [103]:
# 检测目标卡牌
target_card = game_auto.detect_target_card(screen)
game_auto.current_state['target_card'] = target_card


检测到目标卡牌: snowflake (置信度: 1.00)


In [104]:
# 检测玩家状态
player_states = game_auto.detect_player_states(screen)
game_auto.current_state['player_states'] = player_states

醉酒匹配置信度:  False 0.27077335119247437
player1 状态: 正常 (置信度: 0.94)
醉酒匹配置信度:  False 0.22407038509845734
player2 状态: 正常 (置信度: 0.97)
醉酒匹配置信度:  False 0.2123434692621231
player3 状态: 正常 (置信度: 0.96)


In [105]:
# 检测是否轮到玩家回合
is_my_turn = game_auto.is_my_turn(screen)
game_auto.current_state['is_my_turn'] = is_my_turn

检测到 0 张手牌


In [ ]:

def main():
    """主程序入口"""
    print("=== 游戏自动化系统 ===")
    
    # 创建游戏自动化实例
    game_auto = GameAutomation()
    
    # 创建模板目录
    game_auto.create_template_directories()
    
    print("\n请确保：")
    print("1. 已将卡牌模板图像放置在 'card_templates' 目录下")
    print("2. 已将玩家状态模板图像放置在 'player_templates' 目录下")
    print("3. 游戏窗口已打开并处于全屏状态")
    print("4. 按 Ctrl+C 可停止程序")
    
    input("\n准备就绪后按回车开始...")
    
    # 开始检测循环
    game_auto.run_detection_loop()

if __name__ == "__main__":
    main()